## Testing LLM and Spotify API

In [1]:
##Imports
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
#from transformers import pipeline
from langchain.llms import HuggingFaceHub
import pandas as pd
import re

In [2]:
##Functions
def get_song_name(response : str) -> str:
    # Process the response to extract only the song name
    # Assuming the format is like: '"Out of Love" is a song by Avicii.'
    start_quote_index = response.find('"')  # Find the index of the first quote
    end_quote_index = response.find('"', start_quote_index + 1)  # Find the index of the closing quote

    if start_quote_index != -1 and end_quote_index != -1:
        # Extract the song name using slicing
        song_name = response[start_quote_index + 1:end_quote_index]
        return song_name
    else:
        return "Song name not found."
    
def get_track_from_spotify(spotify_client_id, spotify_client_secret, song_name : str) -> str:
    auth_manager = SpotifyClientCredentials(client_id=spotify_client_id, client_secret=spotify_client_secret)
    sp = spotipy.Spotify(auth_manager=auth_manager)
    # Search for the song
    results = sp.search(q=song_name, type='track', limit=1)

    # Extract track information
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        return(f"Found track: {track['name']} by {track['artists'][0]['name']}")
    else:
        return("Song not found.")
    
def prompt_llm(huggingface_api_token, prompt : str, prompt_template : PromptTemplate = None) -> str:
    llm = HuggingFaceHub(
    repo_id="ibm-granite/granite-3.0-3b-a800m-instruct",  # replace with the model name you want
    huggingfacehub_api_token=huggingface_api_token
    )
    if prompt_template:
        llm_chain = LLMChain(llm= llm, prompt= prompt_template)
        response = llm_chain.run(prompt)
    else:
        response = llm(prompt)

    return response

def parse_model_output_to_dict(model_output):
    # Split the model output into lines and create a dictionary from it
    output_lines = model_output.strip().splitlines()
    result_dict = {}
    
    for line in output_lines:
        # Use regex to extract the field name and its numeric value
        match = re.match(r"(\w+):\s([\d.]+)", line)
        if match:
            key, value = match.groups()
            # Convert numeric values appropriately (int or float based on context)
            result_dict[key] = float(value) if '.' in value else int(value)
    
    return result_dict

def parse_model_output_to_dataframe(model_output):
    # Use the dictionary function to parse the output first
    result_dict = parse_model_output_to_dict(model_output)
    
    # Convert dictionary to DataFrame with one row
    result_df = pd.DataFrame([result_dict])
    
    return result_df

# def get_track_recommendations(spotify_client_id, spotify_client_secret, parsed_data):
#     auth_manager = SpotifyClientCredentials(client_id=spotify_client_id, client_secret=spotify_client_secret)
#     sp = spotipy.Spotify(auth_manager=auth_manager)
#     # Prepare the input parameters for the Spotify API
#     recommendations = sp.recommendations(
#         seed_tracks=[],  # Specify your seed tracks if any
#         acousticness=parsed_data['acousticness'],
#         danceability=parsed_data['danceability'],
#         duration_ms=parsed_data['duration'],  # Duration in milliseconds
#         energy=parsed_data['energy'],
#         instrumentalness=parsed_data['instrumentalness'],
#         key=parsed_data['key'],
#         popularity=parsed_data['popularity'],  # Note: Popularity is not a direct filter
#         speechiness=parsed_data['speechiness'],
#         tempo=parsed_data['tempo'],
#         valence=parsed_data['valence'],
#         limit=10  # Number of recommendations to return
#     )

    return recommendations

def get_track_recommendations(spotify_client_id, spotify_client_secret, parsed_data, seed_tracks=None, seed_artists=None, seed_genres=None, country=None, limit=10):
    auth_manager = SpotifyClientCredentials(client_id=spotify_client_id, client_secret=spotify_client_secret)
    sp = spotipy.Spotify(auth_manager=auth_manager)
    # Ensure at least one of the seed parameters is provided
    if not (seed_tracks or seed_artists or seed_genres):
        raise ValueError("At least one of seed_tracks, seed_artists, or seed_genres must be provided.")

    # Prepare the input parameters for the Spotify API
    try:
        recommendations = sp.recommendations(
            seed_tracks=seed_tracks,
            seed_artists=seed_artists,
            seed_genres=seed_genres,
            limit=limit,
            country=country,
            target_acousticness=parsed_data['acousticness'],
            target_danceability=parsed_data['danceability'],
            target_duration_ms=parsed_data['duration'],
            target_energy=parsed_data['energy'],
            target_instrumentalness=parsed_data['instrumentalness'],
            target_key=parsed_data['key'],
            target_speechiness=parsed_data['speechiness'],
            target_tempo=parsed_data['tempo'],
            target_valence=parsed_data['valence']
        )
        return recommendations
    except spotipy.SpotifyException as e:
        print(f"Error: {e}")
        return None

In [3]:

# Replace these with your actual Spotify API credentials
client_id = os.getenv('SPOTIFY_CLIENT_ID')
client_secret = os.getenv('SPOTIFY_CLIENT_SECRET')

huggingface_api_token = os.getenv('HUGGING_FACE_TOKEN')

## SPOTIFY CLASS

In [4]:
class SpotifyAPI:
    
    sp : spotipy.Spotify
    
    #Constructor. Takes SpotifyAPI credentials.
    def __init__(self, client_id, secret_id):
        auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=secret_id)
        self.sp = spotipy.Spotify(auth_manager=auth_manager)
        
    #The get seed methods are useful to get the seed code from a name in the case of tracks and artists.
    #Also for checking if the genre name provided is available in the Spotify API.
    def get_track_seed(self, track_name : str) -> str:
        if not track_name:
            return ''
        else:
            result = self.sp.search(q=track_name, type='track', limit=1)
            if result['tracks']['items']:
                track_id = result['tracks']['items'][0]['id']
                return track_id
            else:
                return "Track not found."
        
    def get_artist_seed(self, artist_name: str) -> str:
        if not artist_name:
            return ''
        else:
            result = self.sp.search(q=artist_name, type='artist', limit=1)
            if result['artists']['items']:
                artist_id = result['artists']['items'][0]['id']
                return artist_id
            else:
                return "Artist not found."
            
    def get_genre_seed(self, genre_name : str) -> str:
        genre_seeds = self.get_all_genre_seed()
        if genre_name in genre_seeds:
            return genre_name
        else:
            return 'Genre not found'
        
    #For autocompletion of the genre input field. May improve user experience
    def get_all_genre_seed(self) -> list:
        genre_seeds = self.sp.recommendation_genre_seeds()
        return genre_seeds['genres']
    
    #Main method for getting recommendations. Returns a list of names (strings) for all songs.
    def get_track_recommendations(self, track_seeds : list[str] = None,\
        artist_seeds : list[str] = None,genre_seeds : list[str] = None, \
            amount : int = '20', country : str = 'US', spotify_data : dict = None) -> list:
        if not (track_seeds or artist_seeds or genre_seeds):
            raise ValueError("At least one of seed_tracks, seed_artists, or seed_genres must be provided.")
        #Debug spotify data
        print('Spotify Data: ', spotify_data)
        
        # Prepare the input parameters for the Spotify API
        try:
            recommendations = self.sp.recommendations(
                seed_tracks=track_seeds,
                seed_artists=artist_seeds,
                seed_genres=genre_seeds,
                limit=amount,
                country=country,
                target_acousticness=spotify_data.get('acousticness', 0.5) if spotify_data else 0.5,
                target_danceability=spotify_data.get('danceability', 0.5) if spotify_data else 0.5,
                target_duration_ms=int(210000) if spotify_data else 210000,  #Replace with an integer
                target_energy=spotify_data.get('energy', 0.5) if spotify_data else 0.5,
                target_instrumentalness=spotify_data.get('instrumentalness', 0.0) if spotify_data else 0.0,
                target_speechiness=spotify_data.get('speechiness', 0.5) if spotify_data else 0.5,
                target_tempo=spotify_data.get('tempo', 120.0) if spotify_data else 120.0,
                target_valence=spotify_data.get('valence', 0.5) if spotify_data else 0.5
            )
            return [track['name'] for track in recommendations['tracks']]
        except spotipy.SpotifyException as e:
            print(f"Error: {e}")
            return None
        
    #Returns the url of the song preview, which is an MP3 file of 30 second duration.
    def get_track_sample(self, track_name : str) -> str:
        track_id = self.get_track_seed(track_name)
        try:
            # Fetch track details from Spotify
            track = self.sp.track(track_id)
            
            # Extract the preview URL if available
            preview_url = track.get('preview_url')
            if preview_url:
                #print(f"Preview URL for track {track_id}: {preview_url}")
                return preview_url
            else:
                #print(f"No preview available for track {track_id}.")
                return None
        except Exception as e:
            print(f"An error occurred: {e}")
            return None

## LLM Class

In [5]:
class LLM:
    
    huggingface_api_token = os.getenv('HUGGING_FACE_TOKEN')
    user_name : str = 'User'
    
    #Templates
    #song_description = 'Upbeat music to dance along'
    
    greet_template = PromptTemplate(
        input_variables=['user_name'],
        template='''Greet a user called: {user_name} in a friendly but professional way.
                                                Please limit your response to a single line of text.
                                                In this context you are a web application that is going to recommend music to the user.
                                                Make the response a little bit different every time.'''
    )
    
    song_template = PromptTemplate(
        input_variables=['description'],
        template='Recommend me a song that matches this description: {song_description}.\
            Include in your response only the name of the song, and only one song.'
    )

    spotify_data_template = PromptTemplate(
        input_variables=['user_input'],
        template='''Given the song description: {user_input}.

Provide specific numeric values for the following Spotify API parameters without any extra text:

acousticness: 
danceability: 
duration: 
energy: 
instrumentalness: 
key: 
popularity: 
speechiness: 
tempo: 
valence: '''
    )
    
    custom_recommendation_template = PromptTemplate(
        input_variables=['user_name'],
        template='''The user's name is: {user_name}. Please let the user know, in a professional and friendly way, that you have a list of songs
        that you would like to recommend to them. In this context you are a web application that is going to recommend music to the user. Please limit your response to a single line of text.
        
        '''
    )
    
    def __init__(self, huggingface_key):
        self.huggingface_api_token = huggingface_key
        pass
    
    def set_username(self, user_name : str) -> None:
        self.user_name = user_name
    
    def prompt_llm(self, prompt : str, prompt_template : PromptTemplate = None) -> str:
        llm = HuggingFaceHub(
        repo_id="ibm-granite/granite-3.0-3b-a800m-instruct",  # replace with the model name you want
        huggingfacehub_api_token=self.huggingface_api_token
        )
        if prompt_template:
            llm_chain = LLMChain(llm= llm, prompt= prompt_template)
            response = llm_chain.run(prompt)
        else:
            response = llm(prompt)

        return response

    def parse_model_output_to_dict(self, model_output):
        # Split the model output into lines and create a dictionary from it
        output_lines = model_output.strip().splitlines()
        result_dict = {}
        
        for line in output_lines:
            # Use regex to extract the field name and its numeric value
            match = re.match(r"(\w+):\s([\d.]+)", line)
            if match:
                key, value = match.groups()
                # Convert numeric values appropriately (int or float based on context)
                result_dict[key] = float(value) if '.' in value else int(value)
        
        return result_dict

    def parse_model_output_to_dataframe(self, model_output):
        # Use the dictionary function to parse the output first
        result_dict = self.parse_model_output_to_dict(model_output)
        
        # Convert dictionary to DataFrame with one row
        result_df = pd.DataFrame([result_dict])
        
        return result_df
    
    ########Pormpt functions##########
    #Returns text greeting the user according to its username. Must call set_username({user's_name}) first
    def greet_user(self) -> str:
        model_response = self.prompt_llm( prompt = self.user_name, prompt_template = self.greet_template)
        #Remove any additional text from the model's response.
        model_response = model_response.replace(f'''Greet a user called: {self.user_name} in a friendly but professional way.
                                                Please limit your response to a single line of text.
                                                In this context you are a web application that is going to recommend music to the user.
                                                Make the response a little bit different every time.''', '')
        return model_response
    
    #Returns a pandas DataFrame containing the additional values needed for a custon Spotify API Recommendations call.
    #The values are listed under the recommendation_template.
    def get_spotify_recommendation_data(self, user_input : str) -> pd.DataFrame:
        model_output = self.prompt_llm( prompt=user_input, prompt_template=self.spotify_data_template)
        return self.parse_model_output_to_dict(model_output=model_output)
    
    #Given a list of song names, this method returns a custom message for the user with his requested recommendations.
    def give_user_recommendations(self, song_names_list : list[str], song_previews : list[str] = None) -> str:
        
        model_output = self.prompt_llm( prompt=self.user_name, prompt_template=self.custom_recommendation_template)
        response = model_output
        #Removes the instructions from the response
        response = model_output.replace(f'''The user's name is: {self.user_name}. Please let the user know, in a professional and friendly way, that you have a list of songs
        that you would like to recommend to them. In this context you are a web application that is going to recommend music to the user. Please limit your response to a single line of text.
        
        ''', '')
        i = 0
        for song_name in song_names_list:
            response += '\n'        
            response += song_name
            if song_previews and song_previews[i] != None:
                response += ' preview url: ' + song_previews[i]
            i += 1   
        
        return response

In [6]:
sp = SpotifyAPI(client_id, client_secret)
llm = LLM(huggingface_api_token)

In [7]:
#Testing the user greeting
llm.set_username('Johnny')
print(llm.greet_user())


C:\Users\JaSON\AppData\Local\Temp\ipykernel_29992\242924301.py:57: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(
C:\Users\JaSON\AppData\Local\Temp\ipykernel_29992\242924301.py:62: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm= llm, prompt= prompt_template)
C:\Users\JaSON\AppData\Local\Temp\ipykernel_29992\242924301.py:63: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm_chain.ru



Greetings, Johnny! I'm here to suggest some tunes that might pique your interest. Let's embark on this musical journey together!


In [8]:
#Testing the user recommendations
llm.set_username('Jefferson')
print(sp.get_all_genre_seed())
print('Genre seed: ', sp.get_genre_seed('blues'))
user_input = 'Workout music. Energetic!'
spotify_data = llm.get_spotify_recommendation_data(user_input=user_input)
recommendations = sp.get_track_recommendations(genre_seeds=['blues'], amount = 10, spotify_data=spotify_data)
samples = []
for song_name in recommendations:
    samples.append(sp.get_track_sample(song_name))
    
print(llm.give_user_recommendations(song_names_list=recommendations, song_previews=samples))

['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient', 'anime', 'black-metal', 'bluegrass', 'blues', 'bossanova', 'brazil', 'breakbeat', 'british', 'cantopop', 'chicago-house', 'children', 'chill', 'classical', 'club', 'comedy', 'country', 'dance', 'dancehall', 'death-metal', 'deep-house', 'detroit-techno', 'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm', 'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk', 'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove', 'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle', 'heavy-metal', 'hip-hop', 'holidays', 'honky-tonk', 'house', 'idm', 'indian', 'indie', 'indie-pop', 'industrial', 'iranian', 'j-dance', 'j-idol', 'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino', 'malay', 'mandopop', 'metal', 'metal-misc', 'metalcore', 'minimal-techno', 'movies', 'mpb', 'new-age', 'new-release', 'opera', 'pagode', 'party', 'philippines-opm', 'piano', 'pop', 'pop-film', 'post-dubstep', 'power-po

Values for different prompts (Using 'blues'):
'Slow-paced music with a calming touch'  ==  {'acousticness': 0.5, 'danceability': 0.2, 'duration': 3.5, 'energy': 0.3, 'instrumentalness': 0.7, 'key': 0.5, 'popularity': 0.3, 'speechiness': 0.1}

'Fast_paced and upbeat music'  ==  {'valence': 0.0, 'acousticness': 0.5, 'danceability': 0.8, 'duration': 2.5, 'energy': 0.8, 'instrumentalness': 0.2, 'key': 0.0, 'popularity': 0.0}

'Sad music for when you are feeling down'  ==  {'acousticness': 0.45, 'danceability': 0.42, 'duration': 3.54, 'energy': 0.58, 'instrumentalness': 0.23}

'Workout music. Energetic!'  ==  {'acousticness': 0.5, 'danceability': 0.7, 'duration': 3.2, 'energy': 0.8, 'instrumentalness': 0.3}